# Bitcoin Data Pipeline
Este notebook contém **ETL completo e análise SQL local** usando DuckDB, em Python

## Instalando duckdb

In [2]:
!pip install duckdb --upgrade -q

## Importar Bibliotecas

In [3]:
import requests
import pandas as pd
from datetime import datetime
import duckdb
import os

## Funções de Extração

In [6]:
def extrair_dados_bitcoin():
    """Extrai preço spot do Bitcoin via API Coinbase"""
    url = 'https://api.coinbase.com/v2/prices/spot'
    resposta = requests.get(url)
    return resposta.json()

def extrair_cotacao_usd_brl():
    """Extrai cotação USD - BRL via API CurrencyFreaks"""
    api_key = '29e51fa03914465ea22f3eda09f4cd9e' 
    url = f'https://api.currencyfreaks.com/v2.0/rates/latest?apikey={api_key}'
    resposta = requests.get(url)
    return resposta.json()

## Função de Transformação

In [7]:
def tratar_dados_bitcoin(dados_json, taxa_usd_brl):
    """Transforma dados da API, converte para BRL e adiciona timestamp"""
    valor_usd = float(dados_json['data']['amount'])
    valor_brl = valor_usd * taxa_usd_brl
    timestamp = datetime.now()
    
    df = pd.DataFrame([{
        "valor_usd": valor_usd,
        "valor_brl": valor_brl,
        "criptomoeda": dados_json['data']['base'],
        "moeda_original": dados_json['data']['currency'],
        "taxa_usd_brl": taxa_usd_brl,
        "timestamp": timestamp
    }])
    
    return df

## ETL - Extração, Transformação e Carga

In [11]:
# Extração
dados_bitcoin = extrair_dados_bitcoin()
cotacao = extrair_cotacao_usd_brl()
taxa_usd_brl = float(cotacao['rates']['BRL'])

# Transformação
df = tratar_dados_bitcoin(dados_bitcoin, taxa_usd_brl)

# Criar pastas se não existirem
os.makedirs('data/raw/json', exist_ok=True)
os.makedirs('data/raw/csv', exist_ok=True)
os.makedirs('data/raw/parquet', exist_ok=True)

# Timestamp para nomes de arquivo
ts = df['timestamp'][0].strftime("%Y%m%d_%H%M%S")

# Carga — salvar arquivos
df.to_json(f'data/raw/json/bitcoin_{ts}.json', orient='records', indent=2, date_format='iso')
df.to_csv(f'data/raw/csv/bitcoin_{ts}.csv', index=False)
df.to_parquet(f'data/raw/parquet/bitcoin_{ts}.parquet', index=False)

print("ETL concluído e arquivos salvos.")

ETL concluído e arquivos salvos.


## Análise SQL Local com DuckDB

In [17]:
# Criar a pasta curated se não existir
os.makedirs('data/curated', exist_ok=True)

# Conectar/ criar DuckDB local
con = duckdb.connect('data/curated/analytics.duckdb')

# Criar tabela com dados Parquet
con.execute("""
CREATE OR REPLACE TABLE bitcoin_data AS
SELECT * FROM read_parquet('data/raw/parquet/*.parquet')
""")

# Analytics
print("Últimos registros:")
display(con.execute("SELECT * FROM bitcoin_data ORDER BY timestamp DESC LIMIT 10").fetchdf())

# Media
print("Média do valor do Bitcoin em BRL:")
display(con.execute("""
SELECT criptomoeda, ROUND(AVG(valor_brl),2) as media_valor_brl
FROM bitcoin_data
GROUP BY criptomoeda
""").fetchdf())

# Valor máximo do Bitcoin em BRL
print("Valor MAX do Bitcoin em BRL:")
max_valor = con.execute("""
SELECT MAX(valor_brl) AS valor_maximo
FROM bitcoin_data
""").fetchdf()
display(max_valor)

# Valor mínimo do Bitcoin em BRL
print("Valor MIN do Bitcoin em BRL:")
min_valor = con.execute("""
SELECT MIN(valor_brl) AS valor_minimo
FROM bitcoin_data
""").fetchdf()
display(min_valor)

# Valor mais recente do Bitcoin (último timestamp)
print("Valor mais recente do Bitcoin em BRL:")
ultimo_valor = con.execute("""
SELECT valor_brl, timestamp
FROM bitcoin_data
ORDER BY timestamp DESC
LIMIT 1
""").fetchdf()
display(ultimo_valor)

Últimos registros:


,valor_usd,valor_brl,criptomoeda,moeda_original,taxa_usd_brl,timestamp
0,86247.165,475157.193776,BTC,USD,5.50925,2025-12-17 19:30:26.501606
1,86250.315,475174.547914,BTC,USD,5.50925,2025-12-17 19:30:16.229972
2,86217.150,474991.833637,BTC,USD,5.50925,2025-12-17 19:30:05.960277
3,86380.825,475893.560131,BTC,USD,5.50925,2025-12-17 19:28:52.292419


Média do valor do Bitcoin em BRL:


,criptomoeda,media_valor_brl
0,BTC,475304.28


Valor MAX do Bitcoin em BRL:


,valor_maximo
0,475893.560131


Valor MIN do Bitcoin em BRL:


,valor_minimo
0,474991.833637


Valor mais recente do Bitcoin em BRL:


,valor_brl,timestamp
0,475157.193776,2025-12-17 19:30:26.501606
